In [2]:
import sqlite3
from datetime import datetime

connection = sqlite3.connect('inventory_db.db')
db_cursor = connection.cursor()

db_cursor.execute('''CREATE TABLE IF NOT EXISTS Products (
                        product_id INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        price REAL NOT NULL,
                        description TEXT)''')

db_cursor.execute('''CREATE TABLE IF NOT EXISTS Inventory (
                        inventory_id INTEGER PRIMARY KEY,
                        product_id INTEGER,
                        quantity INTEGER NOT NULL,
                        FOREIGN KEY(product_id) REFERENCES Products(product_id))''')

db_cursor.execute('''CREATE TABLE IF NOT EXISTS Sales (
                        sale_id INTEGER PRIMARY KEY,
                        product_id INTEGER,
                        sold_quantity INTEGER NOT NULL,
                        sale_date TEXT NOT NULL,
                        FOREIGN KEY(product_id) REFERENCES Products(product_id))''')

connection.commit()

def add_product(name, price, description):
    db_cursor.execute('''INSERT INTO Products (name, price, description)
                         VALUES (?, ?, ?)''', (name, price, description))
    connection.commit()

def update_product(product_id, name=None, price=None, description=None):
    if name:
        db_cursor.execute('''UPDATE Products SET name = ? WHERE product_id = ?''', (name, product_id))
    if price:
        db_cursor.execute('''UPDATE Products SET price = ? WHERE product_id = ?''', (price, product_id))
    if description:
        db_cursor.execute('''UPDATE Products SET description = ? WHERE product_id = ?''', (description, product_id))
    connection.commit()

def delete_product(product_id):
    db_cursor.execute('''DELETE FROM Products WHERE product_id = ?''', (product_id,))
    connection.commit()

def add_inventory(product_id, quantity):
    db_cursor.execute('''INSERT INTO Inventory (product_id, quantity)
                         VALUES (?, ?)''', (product_id, quantity))
    connection.commit()

def update_inventory(inventory_id, quantity):
    db_cursor.execute('''UPDATE Inventory SET quantity = ? WHERE inventory_id = ?''', (quantity, inventory_id))
    connection.commit()

def delete_inventory(inventory_id):
    db_cursor.execute('''DELETE FROM Inventory WHERE inventory_id = ?''', (inventory_id,))
    connection.commit()

def record_sale(product_id, sold_quantity):
    sale_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    db_cursor.execute('''INSERT INTO Sales (product_id, sold_quantity, sale_date)
                         VALUES (?, ?, ?)''', (product_id, sold_quantity, sale_date))
    db_cursor.execute('''UPDATE Inventory SET quantity = quantity - ? WHERE product_id = ?''',
                      (sold_quantity, product_id))
    connection.commit()

def generate_inventory_report():
    db_cursor.execute('''SELECT Products.name, Inventory.quantity
                         FROM Products
                         INNER JOIN Inventory ON Products.product_id = Inventory.product_id''')
    report = db_cursor.fetchall()
    for row in report:
        print(f'{row[0]}: {row[1]} units in stock')

def generate_sales_report():
    db_cursor.execute('''SELECT Products.name, Sales.sold_quantity, Sales.sale_date
                         FROM Products
                         INNER JOIN Sales ON Products.product_id = Sales.product_id''')
    report = db_cursor.fetchall()
    for row in report:
        print(f'{row[0]}: {row[1]} units sold on {row[2]}')

add_product("Tablet", 300.0, "10-inch tablet")
add_product("Keyboard", 45.0, "Mechanical keyboard")

update_product(1, price=280.0)

add_inventory(1, 100)
add_inventory(2, 150)

record_sale(1, 20)

generate_inventory_report()
generate_sales_report()


Tablet: 80 units in stock
Keyboard: 150 units in stock
Tablet: 20 units sold on 2024-08-25 15:37:47
